<center><h1>Datos Meteorológicos con Métodos de Interpolación</h1></center>  
<center><img src="clima.png" width=270 height=270></center>  

### **Sección: 305C1**
### **Estudiantes:**
Manuel Nava 30.822.007  
Juan Wu 30.391.117  
José Farrauto 30.696.288  

---

## Interpolación de Taylor

---

## Interpolación de Lagrange

In [41]:
#=Importar libreria necesaria=#
using PyCall
using Plots
np = pyimport("numpy")

#=Datos de prueba=#
baja = np.array([10,7,4,3,5,2,2]) #=Temperatura baja=# 
alta = np.array([21,18,12,11,14,13,10]) #=Temperatura alta=#

#=Variables para trabajar:=#
dias = length(baja) #Esto es para contar cuanto dias
promedio_T = np.zeros(dias)
dias_lista = np.zeros(dias)

#Sacar promedio de temperatura
for i in 1:dias
    promedio_T[i]= (alta[i]+baja[i])/2
end

#=Contar cantidad de dias=#
j = 1
for i in 1:dias
    dias_lista[i] = j
    j = j + 1
end

#Procedimiento para el Interpolacion de Lagrange:

#Sacar el valor el vector "X":
menordias = np.min(dias_lista)
mayordias = np.max(dias_lista)
xpp = np.linspace(menordias, mayordias, 100)

#Sacar el valor el vector "Y":
ypp = 0
for i in 1:dias
    acum_p = 1 #Para acumular el valor de y
    for j in 1:dias
        if i != j
            acum_p = acum_p*((xpp - dias_lista[j] )/(dias_lista[i]- dias_lista[j]))
        end
    end#=
    ypp += promedio_T[i] * acum_p=#
end

MethodError: MethodError: no method matching -(::Vector{Float64}, ::Float64)
For element-wise subtraction, use broadcasting with dot syntax: array .- scalar

Closest candidates are:
  -(!Matched::PyObject, ::Any)
   @ PyCall C:\Users\Cliente\.julia\packages\PyCall\1gn3u\src\pyoperators.jl:13
  -(::Any, !Matched::PyObject)
   @ PyCall C:\Users\Cliente\.julia\packages\PyCall\1gn3u\src\pyoperators.jl:14
  -(!Matched::BigFloat, ::Union{Float16, Float32, Float64})
   @ Base mpfr.jl:506
  ...


---

## Interpolación de Hermite

---

## Interpolación Polinómicas a Trozos

<center><h2>Gracias!!</h2></center>